In [2]:
import numpy as np
import pandas as pd
from bokeh.plotting import figure
from bokeh.io import output_notebook, show
from bokeh.resources import INLINE
output_notebook()

Loading BokehJS ...

In [69]:
from bokeh.layouts import row,column,gridplot,layout
from bokeh.models.widgets import (
    TextInput, Button, Div, DatePicker, Slider, RadioButtonGroup, Select)
from bokeh.models import ColumnDataSource


In [4]:
def return_calculation(doc):
    
    # create some widgets
    button = Button(label="Submit", button_type="success")
    buy_value = TextInput(title="Buy Value", value='2500')
    buy_date = DatePicker(title='Buy Date', value='2019-01-01')
    sell_value = TextInput(title='Sell Value', value='2620')
    sell_date = DatePicker(title='Sell Date', value='2019-03-25')
    qty = TextInput(title='Quantity', value='100')
    output = Div()

    # add a callback to a widget
    def update():
        bv = float(buy_value.value)
        sv = float(sell_value.value)
        q = int(qty.value)
        start = pd.to_datetime(buy_date.value)
        end = pd.to_datetime(sell_date.value)
        period = (end - start).days
        profit = q*(sv-bv)
        ret = profit/bv
        ann_ret = ret * (365/period)
        text = """
        Net profit = {profit} <br/>
        Holding period = {days} days <br/>
        Return = {ret}% <br/>
        Annualized return = {ann_ret}%
        """
        output.text = text.format(profit=profit, days=period,
                                  ret=round(ret, 2), 
                                  ann_ret=round(ann_ret, 2))
    
    button.on_click(update)

    # create a layout for everything
    l = column(
        row(buy_value, buy_date),
        row(sell_value, sell_date),
        row(qty,button),
        output
    )

    # add the layout to curdoc
    doc.add_root(l)
    
show(return_calculation)

In [6]:
def samples(mu=0.01, sigma=0.01, size=10):
    """
    Return samples from a lognormal distribution
    """
    return np.random.lognormal(mu, sigma, size)

In [80]:
np.random.choice(range(10), 4)

array([2, 7, 4, 3])

In [111]:
def drawdown_effect(doc):
    
    # create some widgets
    button = Button(label="Submit", button_type="success")
    returns = Slider(start=-20,end=30,value=2,step=0.25,
                     title='Returns in percentage')
    label = Div(text="Volatility")
    volatility =  RadioButtonGroup(
        labels=["Very Low", "Low", "Medium", "High", "Very High"],
        active=2)
    periods = Slider(start=1, end=200, value=20, step=1,
                    title="Number of periods")  
    src = ColumnDataSource({'period': [], 'value':[]})

        
    # add a callback to a widget
    def update():
        mu = float(returns.value)*0.01
        sigma = (int(volatility.active)+1)*mu
        size = int(periods.value)
        s = samples(mu=mu, sigma=sigma, size=size)
        src.data = {'period': range(size), 'value': s.cumprod()}
        figs[0].line('period', 'value', source=src)
        for i in range(1,5):
            r = np.random.choice(range(size))
            figs[i].line(range(r,size), s[r:].cumprod())
            
    figs = []
    for i in range(4):
        figs.append(figure())
        
    button.on_click(update)
    
    
    grid = gridplot([fig for fig in figs], ncols=2,
                   plot_width=300, plot_height=250)
    l = layout([
        [returns, periods],
        [label, volatility],
        [button],
        [grid]
        
    ])
    
    # add the layout to curdoc
    doc.add_root(l)
    
show(drawdown_effect)